In [2]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

# Generating SQL queries from natural language

## Loading / creating a dataset

In [3]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

def insert_rows_into_table(rows, table, engine=engine):
    for row in rows:
        stmt = insert(table).values(**row)
        with engine.begin() as connection:
            connection.execute(stmt)

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
insert_rows_into_table(rows, receipts)

## Extracting the table description

The tool’s description attribute will be embedded in the LLM’s prompt by the agent system: it gives the LLM information about how to use the tool. This is where we want to describe the SQL table.

In [4]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


## Creating the SQL tool

In [5]:
from smolagents import tool

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

Now let us create an agent that leverages this tool.

We use the `CodeAgent`, which is smolagents’ main agent class: an agent that writes actions in code and can iterate on previous output according to the ReAct framework.

The model is the LLM that powers the agent system. `InferenceClientModel` allows you to call LLMs using HF’s Inference API, either via Serverless or Dedicated endpoint, but you could also use any proprietary API.

In [6]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="meta-llama/Meta-Llama-3.1-8B-Instruct"),
)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Meta-Llama-3.1-8B-Instruct ──────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  max_price = max(float(price) for price in sql_engine("SELECT price FROM receipts").split(','))                   
  print("Maximum price:", max_price)                                                                               
                                                                                                                   
  max_receipt = sql_engine(f"SELECT * FROM receipts WHERE price = {max_price}")                                    
  print("Maximum receipt data:", max_receipt)                                                                      
                                                                                                                   
  client_name = max_receipt.split()[1]                                                                             
  final_answer(client_name)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'max_price = max(float(price) for price in sql_engine("SELECT price FROM 
receipts").split(','))' due to: ValueError: could not convert string to float: '\n(12.06'

[Step 1: Duration 11.68 seconds| Input tokens: 2,079 | Output tokens: 167]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  max_receipt_data = sql_engine("SELECT customer_name, price FROM receipts ORDER BY price DESC LIMIT 1")           
  print("Maximum receipt data:", max_receipt_data)                                                                 
                                                                                                                   
  client_name = max_receipt_data.split()[0]                                                                        
  final_answer(client_name)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Maximum receipt data: 
('Woodrow Wilson', 53.43)

Out - Final answer: ('Woodrow

[Step 2: Duration 7.99 seconds| Input tokens: 4,534 | Output tokens: 284]

"('Woodrow"

## Level 2: Table joins

Now let’s make it more challenging! We want our agent to handle joins across multiple tables.

So let’s make a second table recording the names of waiters for each receipt_id!

In [7]:
table_name = "waiters"
waiters = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
insert_rows_into_table(rows, waiters)

Since we changed the table, we update the `SQLExecutorTool` with this table’s description to let the LLM properly leverage information from this table.

In [8]:
updated_description = """Allows you to perform SQL queries on the table. 
Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. 
Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


Since this request is a bit harder than the previous one, we’ll switch the LLM engine to use the more powerful [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct)!

In [9]:
sql_engine.description = updated_description

agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct"),
    additional_authorized_imports=["pandas"]
)

agent.run("Which waiter got more total money from tips?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which waiter got more total money from tips?                                                                    │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Fetch tips data                                                                                                
  tips_data = sql_engine(query="SELECT receipt_id, tip FROM receipts;")                                            
  print("Tips Data:", tips_data)                                                                                   
                                                                                                                   
  # Fetch waiters data                                                                                             
  waiters_data = sql_engine(query="SELECT receipt_id, waiter_name FROM waiters;")                                  
  print("Waiters Data:", waiters_data)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Tips Data: 
(1, 1.2)
(2, 0.24)
(3, 5.43)
(4, 1.0)
Waiters Data: 
(1, 'Corey Johnson')
(2, 'Michael Watts')
(3, 'Michael Watts')
(4, 'Margaret James')

Out: None

[Step 1: Duration 10.88 seconds| Input tokens: 2,120 | Output tokens: 178]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Manually parse the fetched data into lists of tuples                                                           
  tips_data = [(1, 1.2), (2, 0.24), (3, 5.43), (4, 1.0)]                                                           
  waiters_data = [(1, 'Corey Johnson'), (2, 'Michael Watts'), (3, 'Michael Watts'), (4, 'Margaret James')]         
                                                                                                                   
  # Create a dictionary to store the total tips for each waiter                                                    
  tips_per_waiter = {}                                                                                             
                                                                                                                   
  # Iterate through the data and calculate the total tips for each waiter                                          
  for tip_entry, waiter_entry in zip(tips_data, waiters_data):                                                     
      receipt_id, tip = tip_entry                                                                                  
      _, waiter_name = waiter_entry                                                                                
      if waiter_name in tips_per_waiter:                                                                           
          tips_per_waiter[waiter_name] += tip                                                                      
      else:                                                                                                        
          tips_per_waiter[waiter_name] = tip                                                                       
                                                                                                                   
  # Determine which waiter got the most tips                                                                       
  waiter_with_most_tips = max(tips_per_waiter, key=tips_per_waiter.get)                                            
  max_tips = tips_per_waiter[waiter_with_most_tips]                                                                
                                                                                                                   
  print(f"The waiter who got the most tips is {waiter_with_most_tips} with {max_tips} tips.")                      
  final_answer(waiter_with_most_tips)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The waiter who got the most tips is Michael Watts with 5.67 tips.

Out - Final answer: Michael Watts

[Step 4: Duration 20.60 seconds| Input tokens: 11,473 | Output tokens: 1,065]

'Michael Watts'